# Registration transform example

This example illustrates how to use a RegistrationTransform to temporally align the frames of an EOPatch using different algorithms

### Create cloudless timelapse

Imports from `sentinelhub` and `eolearn` to set up workflow that creates a timelapse

In [1]:
import datetime
import numpy as np

from sentinelhub import BBox, CRS, MimeType, CustomUrlParam

from eolearn.mask import AddCloudMaskTask, get_s2_pixel_cloud_detector
from eolearn.core import EOPatch, FeatureType, LinearWorkflow
from eolearn.features import SimpleFilterTask
from eolearn.io import S2L1CWCSInput

Set up BBox of ROI and time interval

In [2]:
INSTANCE_ID = None

roi_bbox = BBox(bbox=[128.689942, 37.656454, 128.722946,  37.677434], crs=CRS.WGS84)
# roi_bbox = BBox(bbox=[-6.57257, 37.2732, -5.728, 36.8549], crs=CRS.WGS84)
time_interval = ('2015-01-01', '2017-03-01')

This predicate function filters the images with a cloud coverage larger than a threshold to ensure images do not contain cloudy pixels

In [3]:
class MaxCCPredicate:
    def __init__(self, maxcc):
        self.maxcc = maxcc

    def __call__(self, img_cm):
        w, h, _ = img_cm.shape
        cc = np.sum(img_cm) / (w * h)
        return cc <= self.maxcc

Tasks of the workflow:
 * download S2 images (all 13 bands)
 * run `s2cloudless` to compute cloud masks
 * filter out images with cloud coverage larger than a given threshold (e.g. 0.05)

In [4]:
wcs_task = S2L1CWCSInput(layer='BANDS-S2-L1C', 
                         resx='10m',
                         resy='10m',
                         time_difference=datetime.timedelta(hours=2))

cloud_classifier = get_s2_pixel_cloud_detector(all_bands=True)
add_clm = AddCloudMaskTask(cloud_classifier, 
                           'BANDS-S2-L1C', 
                           cm_size_y='20m',
                           cm_size_x='20m', 
                           cmask_feature='clm', 
                           cprobs_feature='clp')

filter_task = SimpleFilterTask((FeatureType.MASK, 'clm'), MaxCCPredicate(maxcc=0.05))

Finished loading model, total used 170 iterations


Build and execute timelapse as chain of transforms

In [5]:
timelapse = LinearWorkflow(wcs_task, add_clm, filter_task)

result = timelapse.execute({
    wcs_task: {
        'bbox': roi_bbox,
        'time_interval': time_interval
    }
})

Get result as an eopatch

In [6]:
eopatch_clean = [result[key] for key in result.keys()][0]

#### Help function to create GIFs

In [7]:
import imageio, os

def make_gif(eopatch, project_dir, filename, fps):
    """
    Generates a GIF animation from an EOPatch.
    """
    with imageio.get_writer(os.path.join(project_dir, filename), mode='I', fps=fps) as writer:
            for image in eopatch:
                writer.append_data(np.array(image[..., [2, 1, 0]], dtype=np.uint8))

Write clean EOPatch to GIF

In [8]:
make_gif(eopatch=eopatch_clean.data['BANDS-S2-L1C'] * 255, project_dir='.', filename='eopatch_clean.gif', fps=3)

## Run registrations

Import registrations

In [9]:
from eolearn.coregistration import ECCRegistration, ThunderRegistration, PointBasedRegistration

The constructor of the Registration objects takes the attribute type, field name and index of the channel to be used for registration, a dictionary specifying the parameters of the registration, and the interpolation method to be applied to the images. The interpolation methods are (NEAREST, LINEAR and CUBIC). Default is CUBIC. A nearest neighbour interpolation is used on ground-truth data to avoid creation of new labels.

### Thunder registration

This algorithm computes translations only between pairs of images, using correlation on the Fourier transforms of the images

In [10]:
coregister_thunder = ThunderRegistration((FeatureType.DATA, 'BANDS-S2-L1C'), channel=2)

eopatch_thunder = coregister_thunder(eopatch_clean)

INFO:eolearn.coregistration.coregistration:ThunderRegistration:This registration does not require parameters


Write result to GIF

In [11]:
make_gif(eopatch=eopatch_thunder.data['BANDS-S2-L1C']*255, project_dir='.', filename='eopatch_thunder.gif', fps=3)

### Enhanced Cross-Correlation in OpenCV

This algorithm uses intensity values to maximise cross-correlation between pair of images. It uses an Euler transformation (x,y translation plus rotation). 

In [12]:
params = {'MaxIters': 200}
coregister_ecc = ECCRegistration((FeatureType.DATA, 'BANDS-S2-L1C'), channel=2, params=params)

eopatch_ecc = coregister_ecc(eopatch_clean)

INFO:eolearn.coregistration.coregistration:ECCRegistration:Params for this registration are:
INFO:eolearn.coregistration.coregistration:				MaxIters: 200


In [13]:
make_gif(eopatch=eopatch_ecc.data['BANDS-S2-L1C']*255, project_dir='.', filename='eopatch_ecc.gif', fps=3)

### Point-Based Registration in OpenCV

Three transformation models are supported for point-based registration, i.e. Euler, PartialAffine and Homography. These methods compute feature descriptors (i.e. SIFT or SURF) of the pair of images to be registered, and estimate a robust transformation using RANSAC to align the matching points. These methods perform poorly compared to the other methods due to the inaccuracies of the feature extraction, point-matching and model fitting. If unplausible transformations are estimated, a warning is issued and an identity matrix is employed instead of the estimated transform. Default parameters are (Model=Euler, Descriptor=SIFT, RANSACThreshold=7.0, MaxIters=1000).

Note: In case the following cell will raise an error

```Python
AttributeError: module 'cv2.cv2' has no attribute 'xfeatures2d'
```

uninstall and reinstall Python package `opencv-contrib-python`

In [14]:
params = {
    'Model': 'Euler',
    'Descriptor': 'SURF',
    'RANSACThreshold': 7.0,
    'MaxIters': 1000
}

coregister_pbased = PointBasedRegistration((FeatureType.DATA, 'BANDS-S2-L1C'), channel=2, params=params)

eopatch_pbased = coregister_pbased(eopatch_clean)

INFO:eolearn.coregistration.coregistration:PointBasedRegistration:Model set to Euler
INFO:eolearn.coregistration.coregistration:PointBasedRegistration:Descriptor set to SIFT
INFO:eolearn.coregistration.coregistration:PointBasedRegistration:RANSAC MaxIters set to 1000
INFO:eolearn.coregistration.coregistration:PointBasedRegistration:RANSAC threshold set to 7.0
INFO:eolearn.coregistration.coregistration:PointBasedRegistration:Params for this registration are:
INFO:eolearn.coregistration.coregistration:				Model: Euler
INFO:eolearn.coregistration.coregistration:				Descriptor: SIFT
INFO:eolearn.coregistration.coregistration:				MaxIters: 1000
INFO:eolearn.coregistration.coregistration:				RANSACThreshold: 7.00


In [15]:
make_gif(eopatch=eopatch_pbased.data['BANDS-S2-L1C']*255, project_dir='.', filename='eopatch_pbased.gif', fps=3)